In [ ]:
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold
from lightgbm import LGBMClassifier
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/최종제출서류/추가제출파일/train.csv')
print(train.shape)

(7297951, 42)


In [ ]:
def print_stats(X, y):
  X = sm.add_constant(X)
  model = sm.OLS(y, X)
  result = model.fit()
  print(result.summary())

X = train.drop(columns=['application_id', 'user_id', 'is_applied', 'gender', 'yearly_income', 'rehabailitation_total', 
                        'personal_rehabilitation_complete_yn', 'houseown_type', 'existing_avg_loan_amt', 'year_income_age',
                        'loan_limit', 'insert_year', 'loan_minute'])
y = train['is_applied']
print_stats(X, y)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:             is_applied   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                 1.592e+04
Date:                Wed, 12 Oct 2022   Prob (F-statistic):               0.00
Time:                        02:49:13   Log-Likelihood:             3.2522e+05
No. Observations:             7297951   AIC:                        -6.504e+05
Df Residuals:                 7297921   BIC:                        -6.500e+05
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [ ]:
# 제외할 변수 : yearly_income, rehabilitation_total, gender, personal_rehabilitation_complete_yn, houseown_type,
#            existing_avg_loan_amt, year_income_age, loan_limit, insert_year, loan_minute
# 실제로 해당 변수들을 제외하고 LGBMClassifier 돌린 결과...

num_seed = 42
num_cv = 5

X_features = train.drop(columns=['application_id', 'user_id', 'is_applied', 'gender', 'yearly_income', 'rehabailitation_total', 
                        'personal_rehabilitation_complete_yn', 'houseown_type', 'existing_avg_loan_amt', 'year_income_age',
                        'loan_limit', 'insert_year', 'loan_minute'])
y_label = train['is_applied']

lgbm = LGBMClassifier(n_estimators=1000, max_depth=6, boosting_type='gbdt', n_jobs=-1, boost_from_average=False, random_state=num_seed)   # boost_from_average=False --> 레이블 값이 불균형한 경우 사용
scores = cross_validate(lgbm, X_features, y_label, scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc'], cv=num_cv)

print(scores)
print('\n\n')
print(f"acc : {np.mean(scores['test_accuracy'])}")
print(f"pre : {np.mean(scores['test_precision'])}")
print(f"rec : {np.mean(scores['test_recall'])}")
print(f"f1 : {np.mean(scores['test_f1'])}")
print(f"roc_auc : {np.mean(scores['test_roc_auc'])}")

{'fit_time': array([459.07201123, 462.8164742 , 459.42016912, 461.64391947,
       448.16228843]), 'score_time': array([70.33975029, 68.26169825, 65.81370687, 71.39765   , 74.72997546]), 'test_accuracy': array([0.94098689, 0.94106496, 0.94062237, 0.9406943 , 0.94101494]), 'test_precision': array([0.58418825, 0.5874618 , 0.56130514, 0.56702974, 0.58132407]), 'test_recall': array([0.09203002, 0.09340883, 0.09390611, 0.09178138, 0.0964603 ]), 'test_f1': array([0.15901036, 0.16118809, 0.16089461, 0.15798996, 0.1654647 ]), 'test_roc_auc': array([0.87823451, 0.87855456, 0.87505514, 0.87579563, 0.8752224 ])}



acc : 0.9408766926346587
pre : 0.5762617999648378
rec : 0.09351732555774056
f1 : 0.1609095442891513
roc_auc : 0.8765724490161146
